# Resonant Cancellation

## Imports and Definitions

In [1]:
import sys
import os
sys.path.append('/home/sivo/Documents/libraries')

import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import gc
import warnings

from glob import glob
from importlib import reload
from scipy.optimize import curve_fit, fsolve, OptimizeWarning
from scipy.signal import savgol_filter
from scipy.constants import physical_constants
from math import isclose

In [2]:
warnings.simplefilter("error", OptimizeWarning)

In [3]:
%load_ext autoreload
%autoreload 2

import axionLib.axionLib as axion